# Why Mean Time To Failure is a poor(lazy) metric for pre-emptive ESP replacement

## Introduction

In oilfields where Electric Submersible Pump(ESP) is the artificial lift technique, the ESP replacement strategy is a key question. Two strategies are typically employed: 
    
    1) pre-emptive - ESP's are changed out before they fail.
    2) reactive - ESP's are changed out once they fail.

There are unique advantages and disadvantages to either of these strategies, making them unique for different oilfields. However, the chosen strategy impacts OPEX, CAPEX, unscheduled deferment, planning stability and availability of long-lead items making it one of the most important decisions in managing an oilfield.

Recently at work a pre-emptive schedule was chosen with mean time to failure (MTTF) used as the metric to decide on pre-emptive ESP replacement. This post argues that MTTF based replacement is sub-optimal and a reliability based approach using failure rate curves and the probability of failure to decide on when to replace ESP's pre-emptively is a better metric to decide on when to replace ESP's.

The code is available on my github here: [github] 

## Exploratory data analysis

In [55]:
import pandas as pd

failure_data_loc = "data\\esp_failures.csv"
failure_data = pd.read_csv(failure_data_loc)
failure_data.sort_values(by="days", inplace=True)

failure_data["ESP"] = [str(v) for v in failure_data["ESP"]]

failure_data.head(5)

,ESP,days
15,15,2
0,0,51
46,46,90
33,33,97
47,47,107


### Runtimes

In [56]:
from bokeh.plotting import figure, show, row
from bokeh.io import output_notebook

output_notebook()


def plot_runtimes(df: pd.DataFrame):
    p = figure(
        x_range=(0, 3000),
        y_range=df["ESP"],
        height=700,
        width=500,
        title="ESP Runtimes",
        toolbar_location=None,
        tools="",
    )
    p.hbar(y=df["ESP"], right=df["days"], width=0.9)
    p.xgrid.grid_line_color = None

    p.xaxis.axis_label = "Runtime [days]"
    p.yaxis.axis_label = "ESP Number"
    return p


runtimes_plot = plot_runtimes(failure_data)
show(runtimes_plot)  # type: ignore

Loading BokehJS ...

The runtime data shows ESP's with *infant failure*. These are failures that happen immediately after replacement. They're unavoidable and are also a remainder that changing ESP's does not lower risk of failures to zero. In fact, there's a risk of infant failure. The data also shows that at ~1000 days the failure rate shows a step change. And finally you have ESP's which have run for ~2500 days.

The Mean Time To Failure for $n$ ESP's is given by

$$ \frac{Runtime_{esp_1} + Runtime_{esp_2} + ... + Runtime_{esp_n}}{n}$$

In [57]:
mean_time_to_failure = failure_data["days"].mean()

For the above data this translates to *$1178.2$ days*.

In [58]:
from bokeh.models import Span

hline = Span(
    location=1178.2,
    dimension="height",
    line_color="red",
    line_width=2,
    line_dash="dashed",
)

runtimes_plot.renderers.extend([hline])
show(runtimes_plot)  # type: ignore

### Survivor plot

In [59]:
import numpy as np

sample_size = len(failure_data)

survivors = np.array(
    [
        ((sample_size - (idx + 1)) / sample_size)
        for idx, _ in enumerate(failure_data["days"])
    ]
)

failure_data["survivors"] = survivors
failure_data.head()

,ESP,days,survivors
15,15,2,0.98
0,0,51,0.96
46,46,90,0.94
33,33,97,0.92
47,47,107,0.90


In [60]:
from typing import Iterable


class CDFPlot:
    def __init__(self, width=800, height=400, title="CDF Plot", **kwargs):
        self.width = width
        self.height = height

        self.title = title
        self.plot = figure(width=width, height=height, title=title, **kwargs)
        self.lines: dict[str, dict] = {}

    def add_line(
        self,
        line_name: str,
        x: Iterable,
        y: Iterable,
        legend_label: str,
        color: str,
        **kwargs,
    ):
        self.plot.line(x, y, color=color, legend_label=legend_label, **kwargs)
        return self.plot

    def add_scatter(
        self, x: Iterable, y: Iterable, legend: str, color: str, size: int = 15
    ):
        self.plot.circle(x, y, fill_color=color, size=size, legend_label=legend)
        return self.plot

In [61]:
cdf_survivial_plot = CDFPlot(
    title="ESP Survival plot", y_range=(0, 1), height=700, width=700
)
cdf_survivial_plot.plot.xaxis.axis_label = "Run Time [days]"
cdf_survivial_plot.plot.yaxis.axis_label = "Population Fraction Survived"


cdf_survivial_plot.add_scatter(
    x=failure_data["days"],
    y=failure_data["survivors"],
    color="green",
    size=7,
    legend="ESP Survivial Data",
)
cdf_survivial_plot.plot.vspan(
    x=[mean_time_to_failure], line_color="red", line_dash="dashed"
)
show(cdf_survivial_plot.plot)  # type: ignore

The survival plot shows at the beginning $100 %$ of ESP survives. Over time as more and more ESP's fail the survivor population drops. But this plot also shows that at $MTTF$ the ESP does not fail immediately. In this case we have ESP's that has survivied 1400 days more than MTTF. So if you replace the ESP at MTTF you're potentially spending money that's not required.

The key question is given a certain runtime what's the probability of failure of ESP?

In [62]:
show(row([runtimes_plot, cdf_survivial_plot.plot]))  # type: ignore

## Weibull Distribution

The time to failure $T$ of an ESP is a random variable. If $T$ has a Weibull distribution, the failure function

$$F(t) = Pr(T<=t) = \begin{cases}
      1 - e^{-(\lambda t)^\alpha}& {\text{for t} \geq 0} \\
      0 & \text{otherwise}
    \end{cases}$$
  
where $\alpha(>0)$ is the *shape* parameter and $\lambda(>0)$ is the *scale* parameter. When $\alpha = 1$, the Weibull distribution reduces to an exponential distribution.

The survivor function or reliability is

$$ R(t) = Pr(T>0) = e^{-(\lambda t)^\alpha} \text{for t > 0}$$

In [63]:
def F_t(x, scale_lambda, shape_alpha):
    param = 1 - np.exp(-1 * (np.power((scale_lambda * x), shape_alpha)))
    return param

The probability density for Weibull distribution is given by

$$f(t) = \begin{cases}
      \alpha \lambda^\alpha t^{\alpha - 1}e^{-(\lambda t)^\alpha}& \text{for t > 0} \\
      0 & \text{otherwise}
    \end{cases}$$

In [64]:
def f_t(x, scale_lambda, shape_alpha):
    param_1 = shape_alpha * np.power(scale_lambda, shape_alpha)
    param_2 = np.power(x, shape_alpha - 1)
    param_3 = np.exp(-1 * np.power((scale_lambda * x), shape_alpha))

    pdf = param_1 * param_2 * param_3
    return pdf

The failure rate function is

$$ z(t) = \frac{f(t)}{R(t)} = \alpha \lambda^\alpha t^{\alpha - 1}$$

In [65]:
def z_t(shape_alpha, scale_lambda, x):
    param_1 = shape_alpha
    param_2 = np.power(scale_lambda, shape_alpha)
    param_3 = np.power(x, shape_alpha - 1)

    failure = param_1 * param_2 * param_3
    return failure

### Weibull curve fit

#### Weibull parameter extraction

In [66]:
import numpy as np
from scipy.optimize import curve_fit


def normalize(x: Iterable):
    max_value = max(x)
    min_value = 0

    values = []
    for value in x:
        result = (value - min_value) / max_value
        values.append(result)
    return np.array(values)


def weibull_reliability_fit(x, l, a):
    reliability = np.exp(-1 * np.power(l * x, a))
    return reliability


def weibull_reliability(x, l, a):
    return np.exp((-1 * np.power(l * x, a)))

In [67]:
popt, pcov = curve_fit(
    weibull_reliability_fit, normalize(failure_data["days"]), survivors
)
lambda_fit, alpha_fit = popt

print(f"Weibull parameters: Lambda={lambda_fit}, Alpha={alpha_fit}")

Weibull parameters: Lambda=1.7140054427137654, Alpha=1.3658421106365282


#### Weibull Reliability fit

In [68]:
from scipy import interpolate

days_normalized = np.arange(0.001, 1.0001, 0.001)
weibull_cdf = weibull_reliability(days_normalized, lambda_fit, alpha_fit)

reliability_fit_ = weibull_cdf
failure_fit_ = 1 - reliability_fit_

f_reliability = interpolate.interp1d(days_normalized, reliability_fit_)
f_failure = interpolate.interp1d(days_normalized, failure_fit_)


In [69]:
cdf_survivial_fit_plot = CDFPlot(title="ESP Survival plot", y_range=(0, 1))
cdf_survivial_fit_plot.plot.xaxis.axis_label = "Run Time [days]"
cdf_survivial_fit_plot.plot.yaxis.axis_label = "Population Fraction Survived"


cdf_survivial_fit_plot.add_scatter(
    x=failure_data["days"],
    y=failure_data["survivors"],
    color="green",
    size=7,
    legend="ESP Survivial Data",
)
cdf_survivial_fit_plot.plot.vspan(
    x=[mean_time_to_failure], line_color="red", line_dash="dashed"
)

cdf_survivial_fit_plot.add_line(
    line_name="survivors",
    x=days_normalized * max(failure_data["days"]),
    y=reliability_fit_,
    legend_label="Weibull Survivor Curve CDF",
    color="red",
)
cdf_survivial_fit_plot.add_line(
    line_name="failures",
    x=days_normalized * max(failure_data["days"]),
    y=failure_fit_,
    legend_label="Weibull Failure Curve CDF",
    color="green",
)

show(cdf_survivial_fit_plot.plot)  # type: ignore


### Weibull Conditional Survivor

For an item with time to failure $T$ that is put into operation at time $t=0$ and is still functioning at time $t$, the probability that the iten of age t survives an additional interval of length x is 

$$R(x|t) = Pr(T > x+t | T>t) = \frac{Pr(T > x + t)}{Pr(T >t)} = \frac{R(x+t)}{R(t)}$$

In [70]:
def probability_calculation(
    t, delta_t, max_runtime, reliability_function, failure_function
):
    dt = t + delta_t

    normalized_t = t / max_runtime
    normalized_dt = dt / max_runtime

    probability = (
        failure_function(normalized_dt) - failure_function(normalized_t)
    ) / reliability_function(normalized_t)
    return probability


In [73]:
failure_probability = probability_calculation(
    mean_time_to_failure, 365, 2075, f_reliability, f_failure
)


In [87]:
runtimes = np.arange(10, 2075, 10)

probabilities = []
for runtime in runtimes:
    if (runtime + 365) < 0.95 * 2075:
        probability = (
            probability_calculation(
                float(runtime), float(365), float(2075), f_reliability, f_failure
            )
            * 100
        )
    else:
        probability = 100
    probabilities.append(probability)


In [88]:
probabilities

[18.153324955291758,
 18.572173798339588,
 18.95647048561977,
 19.31558579373939,
 19.65466572525577,
 19.976993531452194,
 20.28495038210477,
 20.58039556914376,
 20.864723359684128,
 21.13905963761013,
 21.404333432636427,
 21.661254113870115,
 21.910565153393936,
 22.152845841695978,
 22.388596918791222,
 22.6182591669744,
 22.84219518595921,
 23.060753253668526,
 23.27428058095157,
 23.483059563647785,
 23.687346407233036,
 23.887369631527214,
 24.083308387700633,
 24.27539590746553,
 24.463817309978264,
 24.648738901861435,
 24.83031470032567,
 25.008664484573533,
 25.183926695260674,
 25.356245329341142,
 25.525736193767585,
 25.69250773497683,
 25.856654699631104,
 26.018252767135163,
 26.177417098624883,
 26.33423569062508,
 26.48878850196907,
 26.641151119507462,
 26.791372798283458,
 26.939533024117633,
 27.08570789573681,
 27.22995844152726,
 27.37234269413051,
 27.512908331076773,
 27.651693075659097,
 27.788769929683355,
 27.924187365776305,
 28.057991105266854,
 28.190224

In [91]:
class ProbabilityPlot:
    def __init__(self, width=800, height=400, title="Probability Plot", **kwargs):
        self.width = width
        self.height = height

        self.title = title
        self.plot = figure(width=width, height=height, title=title, **kwargs)
        self.lines: dict[str, dict] = {}

    def add_line(
        self,
        line_name: str,
        x: Iterable,
        y: Iterable,
        legend_label: str,
        color: str,
        **kwargs,
    ):
        self.plot.line(x, y, color=color, legend_label=legend_label, **kwargs)
        return self.plot


In [107]:
probability_plot = ProbabilityPlot(
    title="ESP Failure proabability within next 365 days",
    y_range=(0, 100),
    height=400,
    width=1000,
)
probability_plot.plot.xaxis.axis_label = "Run Time [days]"
probability_plot.plot.yaxis.axis_label = "Probability of Failure"


probability_plot.add_line(
    line_name="probability",
    x=runtimes,
    y=probabilities,
    color="green",
    legend_label="Probability of failure within next 365 days",
)

probability_plot.plot.line(x=list(runtimes), y=probabilities)
probability_plot.plot.vspan(
    x=[mean_time_to_failure], line_color="red", line_dash="dashed", legend_label="MTTF"
)
show(probability_plot.plot)  # type: ignore


The above plot shows the probability of ESP failure within the next 365 days for a given runtime. Replacing an ESP with a new one lowers the probability of failure to ~19%. With increase in runtime, the failure probability gradually goes up. At MTTF the probability of failure is only 36% or there's a 64% chance the ESP will survive for anonther year.

The graph also shows the point at which the ESP will fail within the next 365 days. For the data shown above that's after 1610 days. This also implies that the ESP can potentially run for another 432 days with limited increase in risk. Beyond runtime of 1610 days the risk increases exponentially that pre-emptive change out is justified.

Hence, it's recommended use probabilities of failure rather than MTTF to decide when to replace an ESP.